In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    roc_auc_score
)
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('best_weight_unet_vgg16_04082025.h5')

# Predict probabilities on the test set
y_pred = loaded_model.predict(X_test)

# Threshold to get binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Flatten for metric calculations
y_test_flat = y_test.flatten()
y_pred_flat = y_pred_binary.flatten()
y_pred_prob_flat = y_pred.flatten()

# Function to compute all metrics
def compute_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    intersection = np.sum(np.logical_and(y_true, y_pred))
    union = np.sum(np.logical_or(y_true, y_pred))
    miou = intersection / union if union > 0 else 0
    return accuracy, f1, precision, recall, miou, roc_auc

# Bootstrap to compute mean and std
n_bootstrap = 100
metrics = []

np.random.seed(42)
for _ in range(n_bootstrap):
    indices = np.random.choice(len(y_test_flat), size=len(y_test_flat), replace=True)
    y_true_sample = y_test_flat[indices]
    y_pred_sample = y_pred_flat[indices]
    y_prob_sample = y_pred_prob_flat[indices]
    metrics.append(compute_metrics(y_true_sample, y_pred_sample, y_prob_sample))

metrics = np.array(metrics)
metric_names = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'mIoU', 'ROC-AUC']

print("Performance Metrics with Mean ± Standard Deviation:")
for i, name in enumerate(metric_names):
    mean = np.mean(metrics[:, i])
    std = np.std(metrics[:, i])
    print(f"{name}: {mean:.4f} ± {std:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_flat, y_pred_flat)
print("\nConfusion Matrix:")
print(conf_matrix)

# Optional: Visualize Confusion Matrix
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Non-Forest', 'Forest'],
            yticklabels=['Non-Forest', 'Forest'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()